In [3]:
from astroquery.gaia import Gaia
import numpy as np
Gaia.login()

INFO: Login to gaia TAP server [astroquery.gaia.core]


User:  lpalaver
Password:  ········


INFO: OK [astroquery.utils.tap.core]
INFO: Login to gaia data server [astroquery.gaia.core]
INFO: OK [astroquery.utils.tap.core]


In [4]:
cd "/mnt/beegfs/scratch/data/Gaia-SDSS/fits"

/mnt/beegfs/scratch/data/Gaia-SDSS/fits


In [ ]:
for i,j in zip(np.arange(0, 2000000000, 100000000)[:-1], np.arange(0, 2000000000, 100000000)[1:]):
    query = f"SELECT G.random_index, G.ra, G.dec, G.phot_g_mean_mag, G.parallax, G.parallax_error, D.*,\
	S.objid, S.type, S.psfmag_u, S.psfmag_g, S.psfmag_r, S.psfmag_i, S.psfmag_z, S.psfmagerr_u, S.psfmagerr_g, S.psfmagerr_r, S.psfmagerr_i, S.psfmagerr_z \
FROM gaiadr3.gaia_source AS G \
JOIN external.gaiaedr3_distance AS D ON G.source_id = D.source_id \
JOIN gaiadr3.sdssdr13_best_neighbour AS BN ON G.source_id = BN.source_id \
JOIN external.sdssdr13_photoprimary AS S ON BN.original_ext_source_id = S.objid \
WHERE G.parallax_over_error > 5 \
	AND G.visibility_periods_used > 10 \
	AND G.ruwe < 1.4 \
	AND G.in_qso_candidates = 'f' \
	AND G.in_galaxy_candidates = 'f' \
	AND G.random_index >= {i} \
	AND G.random_index <  {j}".format(i, j)
    print(i, j, "\n", query, "\n")
    job = Gaia.launch_job_async(query, dump_to_file=True, output_format='fits')

0 100000000 
 SELECT G.random_index, G.ra, G.dec, G.phot_g_mean_mag, G.parallax, G.parallax_error, D.*,	S.objid, S.type, S.psfmag_u, S.psfmag_g, S.psfmag_r, S.psfmag_i, S.psfmag_z, S.psfmagerr_u, S.psfmagerr_g, S.psfmagerr_r, S.psfmagerr_i, S.psfmagerr_z FROM gaiadr3.gaia_source AS G JOIN external.gaiaedr3_distance AS D ON G.source_id = D.source_id JOIN gaiadr3.sdssdr13_best_neighbour AS BN ON G.source_id = BN.source_id JOIN external.sdssdr13_photoprimary AS S ON BN.original_ext_source_id = S.objid WHERE G.parallax_over_error > 5 	AND G.visibility_periods_used > 10 	AND G.ruwe < 1.4 	AND G.in_qso_candidates = 'f' 	AND G.in_galaxy_candidates = 'f' 	AND G.random_index >= 0 	AND G.random_index <  100000000 

100000000 200000000 
 SELECT G.random_index, G.ra, G.dec, G.phot_g_mean_mag, G.parallax, G.parallax_error, D.*,	S.objid, S.type, S.psfmag_u, S.psfmag_g, S.psfmag_r, S.psfmag_i, S.psfmag_z, S.psfmagerr_u, S.psfmagerr_g, S.psfmagerr_r, S.psfmagerr_i, S.psfmagerr_z FROM gaiadr3.gaia_sour

In [5]:
from dask.distributed import Client
from hats_import.pipeline import pipeline_with_client
from hats_import.catalog.arguments import ImportArguments

In [6]:
client = Client(n_workers=96, threads_per_worker=1, memory_limit="15GiB")
client

/nvme/lovro/miniconda3/envs/photod-testing/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46547 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:46547/status,
Dashboard: http://127.0.0.1:46547/status,Workers: 96
Total threads: 96,Total memory: 1.41 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44843,Workers: 96
Dashboard: http://127.0.0.1:46547/status,Total threads: 96
Started: Just now,Total memory: 1.41 TiB
Comm: tcp://127.0.0.1:43067,Total threads: 1
Dashboard: http://127.0.0.1:36811/status,Memory: 15.00 GiB
Nanny: tcp://127.0.0.1:46167,


In [7]:
args = ImportArguments(
    sort_columns="source_id",
    ra_column="ra",
    dec_column="dec",
    input_path="/mnt/beegfs/scratch/data/Gaia-SDSS/fits/",
    file_reader="fits",
    output_artifact_name="Gaia-SDSS",
    output_path="/mnt/beegfs/scratch/data/Gaia-SDSS/hats/"
)

In [8]:
pipeline_with_client(args, client)

Planning  :   0%|          | 0/4 [00:00<?, ?it/s]

Mapping   :   0%|          | 0/19 [00:00<?, ?it/s]

Binning   :   0%|          | 0/2 [00:00<?, ?it/s]

Splitting :   0%|          | 0/19 [00:00<?, ?it/s]

Reducing  :   0%|          | 0/63 [00:00<?, ?it/s]

Finishing :   0%|          | 0/4 [00:00<?, ?it/s]